In [1]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import datetime
from datetime import datetime
import re

### cargar archivo 1

In [2]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            padon_1 = pd.read_excel(ruta_archivo, engine='openpyxl',sheet_name=1)
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")Z
            print(padon_1.dtypes)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='G:/Mi unidad/osoetsylra/importaciones_TIS_SD_ETC_a_sidefi/TIS/Padron completo tintoreros 11-23 Actual.xlsx' mode='r' encoding='cp1252'>
CUIL  TITULAR                   int64
CUIL FAMILIAR                 float64
APELLIDO                       object
NOMBRE                         object
DOMICILIO (calle)*             object
Domicilio (altura)*            object
CODIGO POSTAL*                  int64
LOCALIDAD*                     object
TELEFONO                       object
Formulario                    float64
FECHA DE ALTA*         datetime64[ns]
FECHA DE NAC.*                 object
NACIONALIDAD*                   int64
Sexo*                          object
Estado Civil*                   int64
Parentesco*                     int64
Sede                          float64
Discapacidad*                  object
Convenio                       object
dtype: object


In [3]:
print(padon_1[['FECHA DE NAC.*','NOMBRE']].loc[padon_1['NOMBRE']=='MARTA ELIDA'])

   FECHA DE NAC.*       NOMBRE
17     19/05/1979  MARTA ELIDA


### cargar archivo 2

In [ ]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            #padron_2 = pd.read_excel(ruta_archivo, engine='xlrd')
            padron_2 = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padron_2)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


#### FIll cuil familiar vacios

In [4]:
padon_1['CUIL FAMILIAR']= padon_1['CUIL FAMILIAR'].fillna(padon_1['CUIL  TITULAR'])

### Normalizar DNI archivo convenio

In [5]:
#armar el camp DNI juntando las los campos cuil titular y cuil familiar
padon_1['dni']=padon_1['CUIL FAMILIAR']
#limpiar el nuevo campo dni dejando solo los numeros 
padon_1['dni']=padon_1['dni'].apply(lambda x: re.sub('[^0-9]', '', str(x)))
#dejar solo los numreros eliminar caracteres 
padon_1['dni2']=padon_1['dni'].apply(lambda x: x[2:10] if len(x)>= 8 else '0')
#extraer el dni del cuil y dejar vacios los q son menores  a 8
padon_1['dni2']=padon_1['dni2'].astype(int)
#pasar el campo a int 

### normalizar fecha

In [6]:
padon_1['FECHA DE NAC.**']=padon_1['FECHA DE NAC.*'].astype(str).str.strip()
padon_1['FECHA DE NAC.*']=padon_1['FECHA DE NAC.*'].astype(str).str.zfill(8)
#padon_1['Fch_Alta_OS.1']=padon_1['Fch_Alta_OS.1'].astype(str).str.zfill(8)
#padon_1['FECHA DE NAC.*']=padon_1['FECHA DE NAC.*'].astype(str)
#convertir fecha. 
#padon_1=funciones.transformar_fecha(padon_1, 'FECHA DE NAC.*')

#Función para normalizar la fecha
#def normalizar_fecha(fecha):
 #   fecha_normalizada = datetime.strptime(fecha, "%d%m%Y").strftime("%d/%m/%Y")
  #  return fecha_normalizada

#Aplicar la función a la columna "fech_nacimiento"
padon_1['FECHA DE NAC.*'] = padon_1['FECHA DE NAC.*'].apply(funciones.transformar_fecha2)
padon_1['FECHA DE NAC.*'] = padon_1['FECHA DE NAC.*'].astype(str)
#padon_1['Fch_Alta_OS.1'] = padon_1['Fch_Alta_OS.1'].apply(funciones.transformar_fecha2)
#padon_1['Fch_Alta_OS.1'] = padon_1['Fch_Alta_OS.1'].astype(str)
#padon_1[['provincia2','localidad']] = padon_1['Localidad'].str.split('-',expand = True, n=1)
print(padon_1['FECHA DE NAC.*'])

0      25/10/1991
1      13/05/2014
2      25/04/2003
3      24/10/1997
4      14/11/2013
          ...    
277    07/11/2005
278    01/08/2017
279    06/05/2009
280    25/03/1990
281    27/03/1995
Name: FECHA DE NAC.*, Length: 282, dtype: object


In [7]:
print(padon_1[['FECHA DE NAC.*','NOMBRE']].loc[padon_1['NOMBRE']=='MARTA ELIDA'])


   FECHA DE NAC.*       NOMBRE
17     19/05/1979  MARTA ELIDA


In [ ]:
padon_1.to_excel(r"C:\Users\zickd\Downloads\Compressed\fechas.xlsx", engine='openpyxl')

### diccionario


In [8]:
# normalizar textos localidad
padon_1['LOCALIDAD*'] = padon_1['LOCALIDAD*'].str.lower()

sedes_localidades={
"berisso":"la plata",
"moreno":"san justo",
"rafael castillo":"san justo",
"lomas de zamora":"quilmes",
"gregorio de laferrere":"san justo",
"yerba buena":"tucuman",
"pilar":"san martin",
"virrey del pino":"san justo",
"gregorio de la ferrere":"san justo",
"san miguel":"san martin",
"villa luzuriaga":"san justo",
"alejandro korn":"quilmes",
"gonzales catan":"san justo",
"jose c paz":"san martin",
"gonzalez catan":"san justo",
"isidro casanova":"san justo",
"san jose":"quilmes",
"boulogne":"san martin",
"moron":"san justo",
"l del mirador":"san justo",
"mar del plata":"mar del plata",
"ezeiza":"quilmes",
"san miguel de tucuman":"tucuman",
"quilmes":"quilmes",
"jose marmol":"zona sur",
"s. s. de jujuy":"cordoba",
"tandil":"mar del plata",
"tigre":"san martin",
"pablo podesta":"san justo",
"la plata":"la plata ",
"lanus este":"quilmes",
"escobar":"san martin",
"florencio varela":"quilmes",
"villa rosa":"san martin",
"merlo":"san justo",
"caba":"chile",
"villa madero":"san justo",
"bernal oeste":"quilmes",
"el palomar":"san justo",
"almirante brown":"quilmes",
"villa celina":"san justo",
"lomas del mirador":"san justo",
"brandsen":"la plata",
"rafael calzada":"quilmes",
"beccar":"san martin",
"lastenia":"tucuman",
"general rodriguez":"san justo",
"hurlingham":"san martin",
"pontevedra":"san justo",
"glew":"quilmes",
"esteban echeverria":"quilmes",
"san justo":"san justo",
"canning":"quilmes",
"don torcuato":"san martin",
"suarez":"san martin",
"loma hermosa":"san martin",
"berazategui":"quilmes",
"g catan":"san justo",
"avellaneda":"quilmes",
"ciudad evita":"san justo",
"colombres (cruz alta)":"tucuman",
"san fernando":"san martin",
"tafi viejo":"tucuman",
"banda rio sali":"tucuman",
"burzaco":"quilmes",
"banfield":"quilmes",
"ciudadela":"san justo",
"trujui":"san justo",
"la union":"quilmes",
"castelar":"san justo",
"jose c  paz":"san martin",
"cruz alta":"tucuman",
"monte grande":"quilmes",
"ramos mejia":"san justo",
"l.mirador":"san justo",
"san francisco solano":"quilmes",
"claypole":"quilmes",
"marcos paz":"san justo",
"el jaguel":"quilmes",
"general san martin":"san martin",
"la tablada":"san justo",
"lanus":"quilmes",
"puerto madryn":"mendoza",
"los pocitos (patagones)":"chile",
"vicente lopez":"san martin",
"ituzaingo":"san justo",
"alderetes":"tucuman",
"cevil redondo":"tucuman",
"ensenada":"la plata",
"villa bosch":"san martin",
"caseros":"san martin",
"balcarce":"mar del plata",
"villa de mayo (malvinas argentinas)":"san martin",
"ranchillos":"tucuman",
"san matin":"san martin",
"la quiaca (jujuy)":"chile",
"parque san martin":"san justo",
"san nicolas":"rosario ",
"v. scasso (gonzalez catan)":"san justo",
"v. scasso (gonzales catan)":"san justo",
"libertad":"san justo",
"los polvorines":"san martin",
"villa dominico":"quilmes",
"nicole":"san justo",
"bella vista":"san martin",
"domselaar":"quilmes",
"wilde":"quilmes",
"mataderos":"san justo",
"villa carmela":"tucuman",
"dock sud":"quilmes",
"villa lugano":"chile",
"san antonio de padua":"san justo",
"guernica":"quilmes",
"cañuelas":"san martin",
"lanus oeste":"quilmes",
"manuel alberti":"san martin",
"jose leon suarez":"san martin",
"mariano acosta":"san justo",
"adolfo sourdeaux":"san martin",
"longchamps":"quilmes",
"gerli":"quilmes",
"v. del pino":"san justo",
"9 de abril":"quilmes ",
"presidente derqui":"san martin",
"vila rosa":"san martin",
"buenos aires":"quilmes",
"la matanza":"san justo",
"v. celina":"san justo",
"l. mirador":"san justo",
"gregorio la ferrere":"san justo",
"virreydelpino":"san justo",
"neuquen":"mendoza",
"villa regina/ rio negro":"mendoza",
"merlo norte":"san justo",
"ingeniero pablo nogues":"san martin",
"merlos gomez":"san justo",
"villa tesei":"san justo",
"el libertador":"san martin",
"pablo nogues":"san martin",
"tortuguitas":"san martin",
"virrel del pino":"san justo",
"remedios de escalada":"quilmes",
"ingeniero adolfo sordeaux":"san martin",
"grand bourg":"san martin",
"ingeniero budge":"quilmes",
"bernal este":"quilmes",
"villa adelina":"san martin",
"barrio santo tomas":"san martin",
"zelaya":"san martin",
"benavidez":"san martin",
" ":"chile",
"":"chile",
"bernal":"quilmes",
"san vicente":"quilmes",
"v.del pino":"san justo",
"v del pino":"san justo",
"malvinas argentinas":"san martin",
"mercado central":"san justo",
"puente ezcurra":"san justo",
"barrio 17 de marzo":"san justo",
"virrreyes":"san martin",
"jose ingenieros":"san ",
"manzanares":"san martin",
"cuidadela":"san justo",
"banda del rio sali":"tucuman",
"rincon de milberg":"san martin",
"martin coronado":"san justo",
"luis guillon":"quilmes",
"william morris":"san martin",
"ministro rivadavia":"quilmes",
"villa gonnet":"quilmes",
"villa lujan":"tucuman",
"villa hidalgo":"san martin",
"ranelagh":"quilmes",
"segurola 1964":"chile",
"sarandi":"quilmes",
"sourdeaux":"san martin",
"el talar":"san martin",
"villa 31":"chile",
"jose c. paz":"san martin",
"san isidro":"san martin",
"misiones":"chile",
"acasusso":"san martin",
"paso del rey":"san justo",
"san martin":"san martin",
"la boca":"chile",
"la reja":"san justo",
"cruz alta-tucuman":"tucuman",
"victoria":"san martin",
"delta tigre":"san martin",
"virry del pino":"san justo",
"v. soldati":"chile",
"floresta":"chile",
"spegazinni":"quilmes",
"matanza":"san justo",
"loma de zamora":"quilmes",
"caba":"chile",
"billinghurst":"san martin",
"loma verde":"san martin",
"matheu":"san martin",
"alberti":"san martin",
"del viso":"san martin",
"jose clemente paz":"san martin",
"capital federal":"chile",
"ciudad jardin del palomar":"san justo",
"campo de mayo":"san martin",
"loma hermosa":"san martin",
"piñeyro":"quilmes",
"santos lugares":"san martin",
"villa vatteone":"quilmes",
"maquinista f savio":"san martin",
"cañuelas":"quilmes",
"cañueas":"quilmes",
"malaver":"san martin",
"loma hermosa":"san martin",
"piñeyro":"zona sur"
}

sede_codigo={
"la plata":"8",
"san justo":"11",
"quilmes":"14",
"tucuman":"13",
"san martin":"12",
"mar del plata":"6",
"zona sur":"14",
"cordoba":"2",
"chile":"10",
"mendoza":"1",
"rosario ":"5"
}

padon_1['LOCALIDAD*']=padon_1['LOCALIDAD*'].str.strip()

In [9]:
padon_1['sede']=padon_1['LOCALIDAD*'].map(sedes_localidades)
padon_1['sede']=padon_1['sede'].map(sede_codigo)
padon_1[['LOCALIDAD*','sede']]


,LOCALIDAD*,sede
0,la tablada,11
1,la tablada,11
2,la tablada,11
3,la tablada,11
4,la tablada,11
...,...,...
277,merlo,11
278,merlo,11
279,merlo,11
280,san miguel de tucuman,13


In [ ]:
padon_1["N° DE FORMULARIO"]=np.nan
padon_1["NACIONALIDAD"]=np.nan
padon_1[["CUIL_Titular","CUIL_Familiar","Apellido","Nombre","Calle","Nro","localidad",
         "Telefono","N° DE FORMULARIO","Fch_Alta_OS.1","Fch_Nac","NACIONALIDAD","Sexo","Estado_Civil",
         "Parentesco","sede","Incapacidad"]].to_excel(r"G:\Mi unidad\osoetsylra\importaciones_TIS_SD_ETC_a_sidefi\meplife\junio\JUNIO_finalv3.xlsx", engine='openpyxl', index =False)

### ERRORES 

In [10]:
condiciones = [
    lambda row: (row['Parentesco*'] != 1 and row['Parentesco*'] != 4 and row['Parentesco*'] != 0) and row['dni2'] >999999 and (pd.to_datetime('today') - pd.to_datetime(row['FECHA DE NAC.*'], format='%d/%m/%Y')).days // 365.25 > 21,
    lambda row: row['dni2'] is None or len(str(row['dni2'])) < 8,
    lambda row: row['FECHA DE NAC.*'] is None or (pd.to_datetime('today') - pd.to_datetime(row['FECHA DE NAC.*'], format='%d/%m/%Y')).days // 365.25 > 80,
    lambda row: row['LOCALIDAD*'] is None

]
mensajes = [
    'Familiar mayor de edad',
    'Error en DNI',
    'Error en la edad',
    'localidad vacía'
]

def generar_columnas_error(row):
    errores = {}
    for i in range(len(condiciones)):
        if condiciones[i](row):
            columna_error = f'error{i+1}'
            errores[columna_error] = mensajes[i]
    if errores:
        return pd.Series(errores)
    else:
        return pd.Series()
    

save_path = filedialog.askdirectory()

df_errores = padon_1.apply(generar_columnas_error, axis=1)
#print(df_errores.head(20))
df_nuevo = pd.concat([padon_1, df_errores],axis=1)
df_sin_errorv1 = df_nuevo[df_errores.any(axis=1)]
df_nuevo.to_excel(save_path + '\padron_total_tis.xlsx', index = False)
df_sin_errorv1.to_excel(save_path + '\sin_erroresv1.xlsx', engine='openpyxl', index=False)

#df_sin_error = df_nuevo[df_nuevo['error1'].notna() | df_nuevo['error2'].notna() | df_nuevo['error3'].notna()]
df_con_error = df_nuevo[df_nuevo.get('error1', pd.Series()).notna() | df_nuevo.get('error2', pd.Series()).notna() | df_nuevo.get('error3', pd.Series()).notna() | df_nuevo.get('error4', pd.Series()).notna()]

df_con_error.to_excel(save_path + '\con_errores.xlsx', engine='openpyxl', index=False)


C:\Users\zickd\AppData\Local\Temp\ipykernel_16944\39142522.py:24: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\zickd\AppData\Local\Temp\ipykernel_16944\39142522.py:24: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\zickd\AppData\Local\Temp\ipykernel_16944\39142522.py:24: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\zickd\AppData\Local\Temp\ipykernel_16944\39142522.py:24: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users

: 

In [ ]:
#df_sin_errores = df_nuevo[df_nuevo['error1'].isnull() & df_nuevo['error2'].isnull() & df_nuevo['error3'].isnull()]

df_sin_errores = df_nuevo[~(df_nuevo.get('error1', pd.Series()).notna() & df_nuevo.get('error2', pd.Series()).notna() & df_nuevo.get('error3', pd.Series()).notna() & df_nuevo.get('error4', pd.Series()).notna())]

#df_errores.to_excel(r'C:\Users\zickd\Downloads\sin_errores.xlsx', engine='openpyxl', index=False)

df_sin_errores.to_excel(save_path + 'sin_erroresv3.xlsx', engine='openpyxl', index=False)

### Merge archivos x DNI

In [ ]:
cruce_tis=pd.merge(padon_1, padron_2, left_on='dni2', right_on='Nro. Documento', how='left', indicator=True)
#cruce_tis.to_excel(r'C:\Users\zickd\Downloads\merge.xlsx', engine='openpyxl', index=False)

cruce_tis_filtro=cruce_tis[cruce_tis['_merge']=='left_only']

#print(cruce_tis_filtro.head(5))

### exportar el cruce

In [ ]:
save_path = filedialog.askdirectory()
#uso iloc por si las columnas tienen el mismo nombre, necesito la primeras 20 columnas
cruce_tis_filtro.iloc[:, :28].to_excel(save_path + '\cruce_meplife.xlsx', engine='openpyxl' ,index = False )



In [ ]:
fecha= '01/01/1974'

fecha_correg= (pd.to_datetime('today') - pd.to_datetime(padon_1['FECHA DE NAC.*'])).days // 365.25 > 50

edad= (pd.to_datetime('today') - pd.to_datetime(fecha)).days // 365.25

print(fecha_correg)
